In [15]:
## Sometimes Jupyter notebook doesn't retain your PATH environment variable -- this will mess up a number of things.
## We recommend specifying the environment variable manually here
import os
import ctypes
os.environ["PATH"] = '/home/mattjones/bin:/home/mattjones/.local/bin:/home/mattjones/myapps/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/home/mattjones/opt/gurobi801/linux64/bin:/home/mattjones/software/bowtie2-2.3.4.2:/home/mattjones/emboss/EMBOSS-6.6.0/emboss'
os.environ['LD_LIBRARY_PATH'] = '/home/mattjones/lib/::/home/mattjones/opt/gurobi801/linux64/lib'

import SingleCellLineageTracing.TreeSolver as ts
from SingleCellLineageTracing.TreeSolver import alleletable_processing as at_process
import pandas as pd 
import numpy as np

import sys

In [16]:
alleletable_fp = "lg_output/test.alleleTable.txt"
alleletable = pd.read_csv(alleletable_fp, sep='\t')
target_lg = 4

### Estimate Allele Proportions

In [ ]:
allele_props = at_process.get_indel_props(alleletable)

### Create character matrix from a lineage group

The first step is to aggregate our observations into a character matrix -- a matrix whose dimensions are `n x m`, where we have `n` cells and `m` characters (total target sites, 3 times the number of integrations in our case). We can use the alleletable_to_character_matrix function here, which takes the following parameters:

- **at**: alleletable (subsetted for a particular lineage group) 
- **out_fp**: the output file path for the resulting character matrix
- **mutation_map**: the alleleproprotions, calculated as above. If nothing is provided, no prior probabilities will be used for later tree reconstructions.
- **old_r**: Use alleles without context (default = False).

This function will write three files: the character matrix, the indel proprotions dictionary, specifying the mutation probabilities for each character to a particular state, and a dictionary translating a state-character pair to an observed allele. The last two files are saved as pickle files, and only written if a mutation_map is provided.

In [ ]:
lg = alleletable[alleletable["lineageGrp"] == target_lg]

at_process.alleletable_to_character_matrix(lg, "test_lg4_character_matrix.txt", mutation_map=allele_props)

### Reconstruct a lineage

You can now reconstruct lineages using the `reconstruct-lineage` command, which takes in many different commands depending on which of many lineage solvers you'd like to use. We provide Neighbor-Joining, Camin-Sokal (implemented with PHYLIP), greedy, Steiner-Tree/ILP, and Hybrid algorithms. Use the `-h` flag to see all possible parameters.

By using one of our algorithms (greedy, hybrid, or ILP), the output will be a networkx object pickle file and a newick text file. The output of the Neighbor-Joining and Camin-Sokal functions is a newick text file 

### Post-Processing a Tree